In [ ]:
# p = np.matmul(U, M)
# df = pd.DataFrame(p)

# df.squeeze()

In [1]:
import numpy as np
from sklearn.model_selection import KFold
import random    
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

learning_rate = 0.0005
num_of_iterations = 75
reguralization_factor = 0.05
num_of_factors = 10

In [2]:
ratings_table = pd.read_csv(filepath_or_buffer='./ml-1m/ratings.dat',
                            sep='::', delimiter=None, header=0, names=['user_id', 'movie_id', 'rating', 'timestamp'] ,engine='python')

# movies_table = pd.read_csv(filepath_or_buffer='./ml-1m/movies.dat',
#                            sep='::', delimiter=None, header=0, engine='python')

# users_table = pd.read_csv(filepath_or_buffer='./ml-1m/users.dat',
#                           sep='::', delimiter=None, header=0, engine='python')

In [25]:
from cmath import isnan

def create_matrices(ratings):
    X = np.array(ratings_table.pivot(index='user_id',columns='movie_id', values='rating'))

    U = np.random.uniform(-0.01, 0.01, (len(X), num_of_factors))
    M = np.random.uniform(-0.01, 0.01, (num_of_factors, len(X[0])))

    return [X, U, M]


In [26]:
def partial_der_reg(error, element):
    deriv_reg = 2*error - reguralization_factor*element
    return deriv_reg


def calculate_predictions(X, U, M):
    previous_rmse = 100000
    
    for _ in range(num_of_iterations):     
        total_errors = []
        for i, xi in enumerate(X):
            for j, xj in enumerate(X[i]):
            
                if isnan(X[i,j]):
                    continue
    
                pred_xij = np.dot(U[i,:],M[:,j].T)
                
                error_xij = X[i,j] - pred_xij
                
                total_errors.append(error_xij)

                for k in range(num_of_factors):
                    Uik = U[i,k] + learning_rate * (partial_der_reg(error_xij, M[k,j]))

                    Mkj = M[k,j] + learning_rate * (partial_der_reg(error_xij, U[i,k]))
                    
                    U[i,k] = Uik
                    M[k,j] = Mkj
            

        rmse = sqrt(sum(total_errors)/len(total_errors))
        
        if rmse == previous_rmse:
            break
        
        previous_rmse = rmse
        
    return [U, M]

In [27]:
X, U, M = create_matrices(ratings_table)

In [ ]:
U, M = calculate_predictions(X, U, M)

In [51]:
p = np.matmul(U, M)
df = pd.DataFrame(p)


,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,-275.533693,-183.067283,-149.844003,-58.380225,-80.415221,-147.890575,-90.652212,-18.629122,-38.663378,-170.826901,...,-26.122907,-5.720354,-40.255690,-59.098522,-8.777318,-179.138638,-27.593970,-8.331865,-4.497864,-54.223150
1,-4425.213054,-2936.970071,-2404.203025,-935.294983,-1291.371684,-2372.120746,-1450.689068,-300.957136,-618.897429,-2742.167745,...,-419.598171,-90.827905,-637.739318,-944.461030,-138.500120,-2873.934911,-441.362500,-140.907325,-73.454512,-872.486124
2,-8710.438012,-5780.827198,-4732.338152,-1841.768975,-2542.200581,-4669.986210,-2856.361708,-591.843732,-1218.104143,-5397.446992,...,-825.284155,-179.491838,-1256.378116,-1859.289600,-273.316943,-5656.776130,-868.696886,-276.739708,-144.252937,-1716.739596
3,-2492.814814,-1653.923694,-1354.180920,-526.611589,-727.488776,-1335.912053,-816.893288,-169.232336,-348.340535,-1544.211736,...,-236.499268,-51.597306,-359.568855,-531.704208,-78.115375,-1618.736015,-248.485135,-79.036414,-41.501546,-491.446341
4,360429.841663,239208.465420,195803.920223,76198.443549,105184.830266,193223.736729,118177.454915,24485.136408,50403.777339,223326.018801,...,34152.071893,7423.346005,51988.502667,76931.695703,11286.358817,234063.304004,35950.058500,11460.556978,5975.661690,71029.932433


In [ ]:

def k_fold_matrix_factorization(data, learning_rate, iterations, regularization, num_factors):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    maes = []
    rmses = []
    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]
        
        X_train, U_train, M_train = create_matrices(train_data)
        
        P_train = calculate_predictions(X_train, U_train, M_train)
        
        X_test, U_test, M_test = create_matrices(test_data)
        
        